# Compare each step of convert_photometry.py to 'phot_to_dframes_breakdown.ipynb' to match outputs for .nwb conversion

### Legend: 

cp = convert_photometry

lv = LabView_photometry_data

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
signals = scipy.io.loadmat('T:/ACh Rats/80B8CE6_ceecee/02222024-L/signals.mat',matlab_compatible=True)

In [3]:
cp_ref = signals["ref"]
cp_sig1 = signals["sig1"]
cp_sig2 = signals["sig2"]

In [4]:
lv_ref = signals["ref"]
lv_sig1 = signals["sig1"]
lv_sig2 = signals["sig2"]

In [5]:
SR = 10000  # sampling rate of photometry system
Fs = 250  # downsample frequency

In [6]:
# CP
cp_green = cp_sig1[0][:: int(SR / Fs)]
cp_reference = cp_ref[0][:: int(SR / Fs)]
cp_red = cp_sig2[0][:: int(SR / Fs)]

In [7]:
# LabView
lv_green = lv_sig1[0][:: int(SR / Fs)]
lv_reference = lv_ref[0][:: int(SR / Fs)]
lv_red = lv_sig2[0][:: int(SR / Fs)]

___________________________________________________________________________________________________

After downsampling, CP data gets converted to pd.Series while LV data gets converted to a DataFrame then each column is saved as a variable

In [8]:
# Convert raw signals to Pandas Series for rolling mean
cp_raw_green = pd.Series(cp_green)
cp_raw_reference = pd.Series(cp_reference)
cp_raw_red = pd.Series(cp_red)

In [9]:
lv_sigs = pd.DataFrame()
lv_sigs['ref'] = lv_reference
lv_sigs['green'] = lv_green
lv_sigs['red'] = lv_red

In [10]:
lv_raw_reference = lv_sigs['ref']
lv_raw_green = lv_sigs['green']
lv_raw_red = lv_sigs['red']

___________________________________________________________________________________________________

In [11]:
smooth_window = int(Fs / 30)

In [12]:
cp_reference = np.array(cp_raw_reference.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(cp_raw_reference), 1)
cp_signal_green = np.array(cp_raw_green.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(cp_raw_green), 1)
cp_signal_red = np.array(cp_raw_red.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(cp_raw_red), 1)

In [13]:
lv_reference = np.array(lv_raw_reference.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(lv_raw_reference), 1)
lv_signal_green = np.array(lv_raw_green.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(lv_raw_green), 1)
lv_signal_red = np.array(lv_raw_red.rolling(window=smooth_window, min_periods=1).mean()).reshape(len(lv_raw_red), 1)

Check if the outputs match

In [14]:
for i, (a, b) in enumerate(zip(cp_reference, lv_reference)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [15]:
for i, (a, b) in enumerate(zip(cp_signal_green, lv_signal_green)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [16]:
for i, (a, b) in enumerate(zip(cp_signal_red, lv_signal_red)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [17]:
plt.figure()
plt.plot(cp_signal_green)
plt.plot(lv_signal_green)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

So far, everything is the same after calculating the rolling average of our signals. Moving on to baseline correction.

In [18]:
def cp_WhittakerSmooth(data, binary_mask, lambda_):
    """
    Penalized least squares algorithm for background fitting

    Wittaker smoothing is a method for fitting a smooth background to noisy data, often used in signal processing to remove baseline noise or trends while preserving the main features of the signal.

    - fits a smooth background to the input data ('data') by minimizaing the sum of squared differences between the data and the background.
    - uses a binary mask ('binary_mask') to identify the signal regions in the data (peaks and troughs) that are not part of the background calculation.
    - uses a penalty term ('lambda_') to control the smoothness of the background. Discourages rapid changes and enforces smoothness.

    input
        data: input data. 1D array that represents the signal data.
        binary_mask: binary mask that defines which points are signals (peaks) and which are background. 0 = Peak / 1 = Background
        lambda_: parameter that can be adjusted by user. The larger lambda is, the smoother the resulting background. Careful: too large can lead to over smoothing.
    output
        the fitted background vector
    """
    data_matrix = np.matrix(data)  # matrix structure of data in order to use matrix operations
    data_size = data_matrix.size  # size of the data matrix
    ID_matrix = eye(
        data_size, format="csc"
    )  # creates an identity matrix the size of data_size in compressed sparse column (csc) format
    diff_matrix = ID_matrix[1:] - ID_matrix[:-1]  # numpy.diff() does not work with sparse matrix. This is a workaround.
    diagonal_matrix = diags(
        binary_mask, 0, shape=(data_size, data_size)
    )  # creates a diagonal matrix with the binary mask values on the diagonal
    A = csc_matrix(
        diagonal_matrix + (lambda_ * diff_matrix.T * diff_matrix)
    )  # represents the combination of the diagonal matrix and the smoothness penalty term Lambda_
    B = csc_matrix(diagonal_matrix * data_matrix.T)  # represents the weighted data
    smoothed_baseline = spsolve(A, B)  # solves the linear system of equations to find the smoothed baseline

    return np.array(smoothed_baseline)


def cp_airPLS(data, lambda_=100, max_iterations=15):
    """
    Adaptive iteratively reweighted penalized least squares for baseline fitting
    DOI : 10.1039/b922045c

    This function is used to fit a baseline to the input data x using adaptive iteratively reweighted penalized least squares.
    The baseline is fitted by minimizing the weighted sum of the squared differences between the input data and the baseline.
    The function uses a penalized least squares approach to fit the baseline, with a penalty term that encourages smoothness.
    The function iteratively adjusts the weights and the baseline until convergence is achieved.

    input
        data: input data (i.e. chromatogram of spectrum)
        lambd: Parameter that can be adjusted by user. The larger lambda is, the smoother the resulting baseline.
        max_iterations: Maximum number of iterations for the algorithm to adjust the weights and fit the baseline.

    output
        the fitted baseline vector
    """

    data_points = data.shape[0]  # number of data points
    weights = np.ones(data_points)  # initial weights set to 1. All points are initially treated equally.

    for i in range(1, max_iterations + 1):
        # loop runs 'max_iterations' times to adjust the weights and fit the baseline
        baseline = cp_WhittakerSmooth(
            data=data, binary_mask=weights, lambda_=lambda_,
        )  
        delta = (
            data - baseline
        )  # difference between data and baseline to calculate residuals, how much each data point deviates from the baseline. delta > 0 == peak. delta < 0 == background.
        sum_of_neg_deltas = np.abs(delta[delta < 0].sum())  # how much data is below the baseline.

        if (
            sum_of_neg_deltas < 0.001 * (abs(data)).sum() or i == max_iterations
        ):  # convergence check: if sum_of_neg_deltas is less than 0.001 of the total data, or if the maximum number of iterations is reached, the loop breaks.
            if i == max_iterations:
                print("WARING max iteration reached!")
            break

        weights[delta >= 0] = (
            0  # delta >= 0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        )
        weights[delta < 0] = np.exp(
            i * np.abs(delta[delta < 0]) / sum_of_neg_deltas
        )  # updates the weights for the negative deltas. Gives more weight to larger residuals using an exponential function.
        weights[0] = np.exp(
            i * (delta[delta < 0]).max() / sum_of_neg_deltas
        )  # updates the weights for the first and last data points to ensure edges of data are not ignored or underweighed.
        weights[-1] = weights[0]
    return baseline

In [19]:
# find baseline for sig and ref

from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve

def lv_WhittakerSmooth(x,w,lambda_,differences=1):
    '''
    Penalized least squares algorithm for background fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        w: binary masks (value of the mask is zero if a point belongs to peaks and one otherwise)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background
        differences: integer indicating the order of the difference of penalties
    
    output
        the fitted background vector
    '''
    X=np.matrix(x)
    m=X.size
    i=np.arange(0,m)
    E=eye(m,format='csc')
    D=E[1:]-E[:-1] # numpy.diff() does not work with sparse matrix. This is a workaround.
    W=diags(w,0,shape=(m,m))
    A=csc_matrix(W+(lambda_*D.T*D))
    B=csc_matrix(W*X.T)
    background=spsolve(A,B)
    return np.array(background)

def lv_airPLS(x, lambda_=100, porder=1, itermax=15):
    '''
    Adaptive iteratively reweighted penalized least squares for baseline fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background, z
        porder: adaptive iteratively reweighted penalized least squares for baseline fitting
    
    output
        the fitted background vector
    '''
    m=x.shape[0]
    w=np.ones(m)
    for i in range(1,itermax+1):
        z=lv_WhittakerSmooth(x,w,lambda_, porder)
        d=x-z
        dssn=np.abs(d[d<0].sum())
        if(dssn<0.001*(abs(x)).sum() or i==itermax):
            if(i==itermax): print('WARING max iteration reached!')
            break
        w[d>=0]=0 # d>0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        w[d<0]=np.exp(i*np.abs(d[d<0])/dssn)
        w[0]=np.exp(i*(d[d<0]).max()/dssn) 
        w[-1]=w[0]
    return z



In [20]:
lambd = 1e8
max_iterations = 50
# smoothed background lines
cp_ref_base = cp_airPLS(cp_raw_reference.T, lambda_=lambd, max_iterations=max_iterations).reshape(len(cp_raw_reference), 1)
cp_g_base = cp_airPLS(cp_raw_green.T, lambda_=lambd,  max_iterations=max_iterations).reshape(len(cp_raw_green), 1)
cp_r_base = cp_airPLS(cp_raw_red.T, lambda_=lambd, max_iterations=max_iterations).reshape(len(cp_raw_red), 1)

In [21]:
lambd = 1e8
porder = 1
itermax = 50
# airPLS Basleine is calculated for photobleaching correction. 
# The baseline represents a drift in the signal (downward decay of fluorescence over time) that is alter used to straighten the signal for clarity. 
# This drift/baseline is identified using a penalized least squares approach
# Minimizes the sum of the squared differences between the observed data and the baseline model and introduces a penalty for the wigglyness of the baseline controlled by the lambda parameter. Larger lambda, more smooth
# Additionally, data points are weighted and refined iteratively where the further they are from the baseline (e.g. a sharp peak), the less weight is given in order to focus more on baseline points 
lv_ref_base=lv_airPLS(lv_raw_reference.T,lambda_=lambd,porder=porder,itermax=itermax).reshape(len(lv_raw_reference),1)
lv_g_base=lv_airPLS(lv_raw_green.T,lambda_=lambd,porder=porder,itermax=itermax).reshape(len(lv_raw_green),1)
lv_r_base=lv_airPLS(lv_raw_red.T,lambda_=lambd,porder=porder,itermax=itermax).reshape(len(lv_raw_red),1)

Compare the baselines that will be subtracted later

In [22]:
for i, (a, b) in enumerate(zip(cp_ref_base, lv_ref_base)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [23]:
for i, (a, b) in enumerate(zip(cp_g_base, lv_g_base)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [24]:
for i, (a, b) in enumerate(zip(cp_r_base, lv_r_base)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [25]:
plt.figure()
plt.plot(cp_g_base)
plt.plot(lv_g_base)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

Everything checks out! Now subtract baseline and recheck for differences

In [26]:
remove = 0
cp_subtracted_reference = cp_reference[remove:] - cp_ref_base[remove:]
cp_subtracted_gsignal = cp_signal_green[remove:] - cp_g_base[remove:]
cp_subtracted_rsignal = cp_signal_red[remove:] - cp_r_base[remove:]

In [27]:
remove = 0
lv_subtracted_reference = lv_reference[remove:] - lv_ref_base[remove:]
lv_subtracted_gsignal = lv_signal_green[remove:] - lv_g_base[remove:]
lv_subtracted_rsignal = lv_signal_red[remove:] - lv_r_base[remove:]

Check for similarity 

In [28]:
for i, (a, b) in enumerate(zip(cp_subtracted_reference, lv_subtracted_reference)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b}")

In [29]:
for i, (a, b) in enumerate(zip(cp_subtracted_gsignal, lv_subtracted_gsignal)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [30]:
for i, (a, b) in enumerate(zip(cp_subtracted_rsignal, lv_subtracted_rsignal)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [31]:
plt.figure()
plt.plot(cp_subtracted_gsignal)
plt.plot(lv_subtracted_gsignal)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

Everything checks out so let's move on to checking if z-scoring the signals are the same for both datasets...

In [32]:
cp_z_reference = (cp_subtracted_reference - np.median(cp_subtracted_reference)) / np.std(cp_subtracted_reference)
cp_z_green = (cp_subtracted_gsignal - np.median(cp_subtracted_gsignal)) / np.std(cp_subtracted_gsignal)
cp_z_red = (cp_subtracted_rsignal - np.median(cp_subtracted_rsignal)) / np.std(cp_subtracted_rsignal)

In [33]:
lv_z_reference = (lv_subtracted_reference - np.median(lv_subtracted_reference)) / np.std(lv_subtracted_reference)
lv_z_green = (lv_subtracted_gsignal - np.median(lv_subtracted_gsignal)) / np.std(lv_subtracted_gsignal)
lv_z_red = (lv_subtracted_rsignal - np.median(lv_subtracted_rsignal)) / np.std(lv_subtracted_rsignal)

In [34]:
for i, (a, b) in enumerate(zip(cp_z_reference, lv_z_reference)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [35]:
for i, (a, b) in enumerate(zip(cp_z_green, lv_z_green)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [36]:
for i, (a, b) in enumerate(zip(cp_z_red, lv_z_red)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [37]:
plt.figure()
plt.plot(cp_z_green)
plt.plot(lv_z_green)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

Yup, still the same. Let's hit it with a lasso regression and calculate dF/F and see whatsup

In [38]:
from sklearn.linear_model import Lasso  # for Lasso regression
lin = Lasso(alpha=0.0001, precompute=True, max_iter=1000, positive=True, random_state=9999, selection="random")

In [39]:
lin.fit(cp_z_reference, cp_z_green)
# lin.fit(cp_z_reference, cp_z_red)
cp_z_reference_fitted = lin.predict(cp_z_reference).reshape(len(cp_z_reference), 1)

In [40]:
lin.fit(lv_z_reference, lv_z_green)
# lin.fit(lv_z_reference, lv_z_red)
lv_z_reference_fitted = lin.predict(lv_z_reference).reshape(len(lv_z_reference), 1)

In [41]:
for i, (a, b) in enumerate(zip(cp_z_reference_fitted, lv_z_reference_fitted)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [42]:
for i, (a, b) in enumerate(zip(cp_z_green, lv_z_green)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

In [43]:
plt.figure()
plt.plot(cp_z_reference_fitted)
plt.plot(lv_z_reference_fitted)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

In [44]:
cp_gzdFF = (cp_z_green - cp_z_reference_fitted)

In [45]:
lv_gzdFF = (lv_z_green - lv_z_reference_fitted)

In [46]:
for i, (a, b) in enumerate(zip(cp_gzdFF, lv_gzdFF)):
    if a != b:
        print(f"Difference found at index {i}: {a} != {b} : difference is {a - b}")

: 

In [95]:
plt.figure()
plt.plot(cp_gzdFF)
plt.plot(lv_gzdFF)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

___________________________________________________________________________________________________